---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('City_Zhvi_AllHomes.csv')
df.head()

RegionID    RegionName State                           Metro    CountyName  \
0      6181      New York    NY                        New York        Queens   
1     12447   Los Angeles    CA  Los Angeles-Long Beach-Anaheim   Los Angeles   
2     17426       Chicago    IL                         Chicago          Cook   
3     13271  Philadelphia    PA                    Philadelphia  Philadelphia   
4     40326       Phoenix    AZ                         Phoenix      Maricopa   

   SizeRank   1996-04   1996-05   1996-06   1996-07   ...     2015-11  \
0         1       NaN       NaN       NaN       NaN   ...      573600   
1         2  155000.0  154600.0  154400.0  154200.0   ...      558200   
2         3  109700.0  109400.0  109300.0  109300.0   ...      207800   
3         4   50000.0   49900.0   49600.0   49400.0   ...      122300   
4         5   87200.0   87700.0   88200.0   88400.0   ...      183800   

   2015-12  2016-01  2016-02  2016-03  2016-04  2016-05  2016-06  2016-07  \
0   576200   578400   582200   588000   592200   592500   590200   588000   
1   560800   562800   565600   569700   574000   577800   580600   583000   
2   206900   206200   205800   206200   207300   208200   209100   211000   
3   121600   121800   123300   125200   126400   127000   127400   128300   
4   185300   186600   188000   189100   190200   191300   192800   194500   

   2016-08  
0   586400  
1   585100  
2   213000  
3   129100  
4   195900  

[5 rows x 251 columns]

In [15]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [16]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt') as file:
        data = []
        for l in file:
            data.append(l[:-1])
    state_town=[]    
    for row in data:
        if row[-6:] == '[edit]':
            state = row[0:-6]
        elif '(' in row:
            town = row[:row.index('(')].strip()
            state_town.append([state,town])
        else:
            town = row
            state_town.append([state,town])
    state_town_df = pd.DataFrame(state_town,columns = ['State','RegionName'])        
    return state_town_df
get_list_of_university_towns().head()

State    RegionName
0  Alabama        Auburn
1  Alabama      Florence
2  Alabama  Jacksonville
3  Alabama    Livingston
4  Alabama    Montevallo

In [20]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp = gdp.iloc[219:]
    gdp.rename(columns= {'Unnamed: 4': 'Quarter' , 'Unnamed: 5' : 'GDP' }, inplace=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    gdp.reset_index(inplace=True)
    gdp.drop(['index'], axis=1,inplace=True)
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] > gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] > gdp.iloc[i][1]):
            return gdp.iloc[i-2][0]
    #return gdp
get_recession_start()

'2008q3'

In [21]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp = gdp.iloc[219:]
    gdp.rename(columns= {'Unnamed: 4': 'Quarter' , 'Unnamed: 5' : 'GDP' }, inplace=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    gdp.reset_index(inplace=True)
    gdp.drop(['index'], axis=1,inplace=True)
    start_rec = gdp[gdp['Quarter'] == get_recession_start()].index.tolist()[0]
    #print(start_rec)
    for i in range(start_rec+1, len(gdp)):
        if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
            return gdp.iloc[i][0]
    #return gdp   
    #return "ANSWER"
get_recession_end()    

'2009q4'

In [30]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
    gdp = gdp.iloc[219:]
    gdp.rename(columns= {'Unnamed: 4': 'Quarter' , 'Unnamed: 5' : 'GDP' }, inplace=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    gdp.reset_index(inplace=True)
    gdp.drop(['index'], axis=1,inplace=True)
    start_rec = gdp[gdp['Quarter'] == get_recession_start()].index.tolist()[0]
    end_rec = gdp[gdp['Quarter'] == get_recession_end()].index.tolist()[0]
    gdp=gdp.iloc[start_rec:end_rec+1]
    bottom = gdp['GDP'].min()
    bottom_index = gdp[gdp['GDP'] == bottom].index.tolist()[0]-start_rec
    #print(bottom_index)
    return gdp.iloc[bottom_index]['Quarter']
get_recession_bottom()

'2009q2'

In [77]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    #dropping not required columns
    df.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=True)
    #print(data.head())
    #rename states
    df.replace({'State':states},inplace=True)  
    #multilevel indexing
    df.set_index(['State','RegionName'],inplace=True)
    col = list(df.columns)
    #print(col.index('2000-01'))
    df.drop(col[0:col.index('2000-01')],inplace=True,axis=1)
    #qts = []
    #for x in range(0, len(list(df.columns)), 3):
    #   qts.append(df.columns[x:x+3].tolist())
    #print(qts)
    qts = [list(df.columns)[x:x+3] for x in range(0, len(list(df.columns)), 3)]
    #print(df[qts[0]].mean(axis=1))
    #new clms
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    #print(quar_years[:-1])

    for col,q in zip(quar_years[0:-1],qts):
        df[col] = df[q].mean(axis=1)
    return df[quar_years[0:-1]]
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State        RegionName                                                  
New York     New York                NaN            NaN            NaN   
California   Los Angeles   207066.666667  214466.666667  220966.666667   
Illinois     Chicago       138400.000000  143633.333333  147866.666667   
Pennsylvania Philadelphia   53000.000000   53633.333333   54133.333333   
Arizona      Phoenix       111833.333333  114366.666667  116000.000000   

                                  2000q4         2001q1         2001q2  \
State        RegionName                                                  
New York     New York                NaN            NaN            NaN   
California   Los Angeles   226166.666667  233000.000000  239100.000000   
Illinois     Chicago       152133.333333  156933.333333  161800.000000   
Pennsylvania Philadelphia   54700.000000   55333.333333   55533.333333   
Arizona      Phoenix       117400.000000  119600.000000  121566.666667   

                                  2001q3         2001q4         2002q1  \
State        RegionName                                                  
New York     New York                NaN            NaN            NaN   
California   Los Angeles   245066.666667  253033.333333  261966.666667   
Illinois     Chicago       166400.000000  170433.333333  175500.000000   
Pennsylvania Philadelphia   56266.666667   57533.333333   59133.333333   
Arizona      Phoenix       122700.000000  124300.000000  126533.333333   

                                  2002q2    ...            2014q2  \
State        RegionName                     ...                     
New York     New York                NaN    ...     515466.666667   
California   Los Angeles   272700.000000    ...     498033.333333   
Illinois     Chicago       177566.666667    ...     192633.333333   
Pennsylvania Philadelphia   60733.333333    ...     113733.333333   
Arizona      Phoenix       128366.666667    ...     164266.666667   

                                  2014q3         2014q4         2015q1  \
State        RegionName                                                  
New York     New York      522800.000000  528066.666667  532266.666667   
California   Los Angeles   509066.666667  518866.666667  528800.000000   
Illinois     Chicago       195766.666667  201266.666667  201066.666667   
Pennsylvania Philadelphia  115300.000000  115666.666667  116200.000000   
Arizona      Phoenix       165366.666667  168500.000000  171533.333333   

                                  2015q2         2015q3         2015q4  \
State        RegionName                                                  
New York     New York      540800.000000  557200.000000  572833.333333   
California   Los Angeles   538166.666667  547266.666667  557733.333333   
Illinois     Chicago       206033.333333  208300.000000  207900.000000   
Pennsylvania Philadelphia  117966.666667  121233.333333  122200.000000   
Arizona      Phoenix       174166.666667  179066.666667  183833.333333   

                                  2016q1         2016q2    2016q3  
State        RegionName                                            
New York     New York      582866.666667  591633.333333  587200.0  
California   Los Angeles   566033.333333  577466.666667  584050.0  
Illinois     Chicago       206066.666667  208200.000000  212000.0  
Pennsylvania Philadelphia  123433.333333  126933.333333  128700.0  
Arizona      Phoenix       187900.000000  191433.333333  195200.0  

[5 rows x 67 columns]

In [92]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df = convert_housing_data_to_quarters().copy()
    start_rec = get_recession_start()
    end_rec = get_recession_end()
    df = df.loc[: , start_rec: end_rec]
    df = df.reset_index()
    #price_ratio
    df['price_ratio'] = (df[start_rec] - df[end_rec])/df[start_rec]
    #university details
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)
    #print(uni_town)
    df['is_uni'] = df.apply(lambda row : row['RegionName'] in uni_town,axis=1)
    not_uni = df[df['is_uni']==False].loc[:,'price_ratio'].dropna()
    is_uni  = df[df['is_uni']==True].loc[:,'price_ratio'].dropna()    
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    if not_uni.mean() < is_uni.mean():
        return (True,p_val,'non-university town')
    else:
        return (True,p_val,'university town')
            
    
run_ttest()

(True, 7.2210481594224543e-05, 'university town')